In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame

In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.5


In [3]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [ ]:
val model = PipelineModel.load(modelPath)
val predictionsDF = model.transform(inputDF)

In [6]:
import org.apache.spark.sql.functions._
val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

model = pipeline_c6fbab19287b
predictionsDF = [tweet: string, words: array<string> ... 4 more fields]
getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


lastException: Throwable = null


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [7]:
// predictionsDF
// .writeStream.
// foreachBatch { (batchDF: DataFrame, batchId: Long) =>
// batchDF.show()
// }.start()

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@20315c72

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Off to run Santa ...|[off, to, run, sa...|(1000,[26,120,173...|[4.85525589321484...|[0.48552558932148...|       1.0|
|I ate too much. N...|[i, ate, too, muc...|(1000,[98,196,329...|[5.62272968131892...|[0.56227296813189...|       0.0|
|3rd june suprise ...|[3rd, june, supri...|(1000,[226,422,78...|[4.85525589321484...|[0.48552558932148...|       1.0|
|@Trucco905 Yes th...|[@trucco905, yes,...|(1000,[48,122,206...|[5.80141001780464...|[0.58014100178046...|       0.0|
|http://www.nytime...|[http://www.nytim...|(1000,[83,114,128...|[5.05140407788463...|[0.50514040778846...|       0.0|
|chemistry tomorro...|[chemistry, tomor...|(1000,[2,35,6

In [8]:
predictionsDF
    .select(
        $"tweet",
        getProbability ($"probability") as ("negative_probability"))
    .writeStream
    .foreachBatch {
        (batchDF : DataFrame, batchId : Long) =>
            batchDF.show ()
}.start ()

+--------------------+--------------------+
|               tweet|negative_probability|
+--------------------+--------------------+
|Off to run Santa ...|  0.5144744106785156|
|I ate too much. N...|  0.4377270318681079|
|3rd june suprise ...|  0.5144744106785156|
|@Trucco905 Yes th...|  0.4198589982195351|
|http://www.nytime...| 0.49485959221153664|
|chemistry tomorro...|  0.5082814404653254|
|summer sucks when...| 0.49432955001111323|
| @erinsunday crappy |  0.5144744106785156|
|@KirbyTheDog uh h...|  0.5282747141644581|
|Is fixing up her ...|  0.5314069222628344|
|@sethsimonds @cop...|  0.5178673482985745|
+--------------------+--------------------+



org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@74cc2f32